In [ ]:
"""
Purpose: To Run the proofreading of neurons in the 
test set and measure the precision and recall of the new rules

Segments to look out for that had no axon
864691135012395766 # one of the split segments files not there (split index = 1)
864691135865584005 #soma soma merger
864691136451113215 #it was an inhibitory neuron (part of split)

"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

INFO - 2021-07-06 21:44:25,458 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-07-06 21:44:25,463 - settings - Setting database.user to celiib
INFO - 2021-07-06 21:44:25,465 - settings - Setting database.password to newceliipass
INFO - 2021-07-06 21:44:25,471 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-07-06 21:44:25,473 - settings - Setting database.user to celiib
INFO - 2021-07-06 21:44:25,475 - settings - Setting database.password to newceliipass
INFO - 2021-07-06 21:44:25,479 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/de

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-07-06 21:44:26,335 - settings - Setting enable_python_native_blobs to True
INFO - 2021-07-06 21:44:26,404 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-07-06 21:44:26,407 - settings - Setting database.user to celiib
INFO - 2021-07-06 21:44:26,411 - settings - Setting database.password to newceliipass
INFO - 2021-07-06 21:44:26,412 - settings - Setting enable_python_native_blobs to True
INFO - 2021-07-06 21:44:27,687 - settings - Setting enable_python_native_blobs to True
INFO - 2021-07-06 21:44:27,862 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-07-06 21:44:27,871 - settings - Setting database.user to celiib
INFO - 2021-07-06 21:44:27,876 - settings - Setting database.password to newceliipass
INFO - 2021-07-06 21:44:27,880 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-07-06 21:44:27,881 - settings - Setting database.user to celiib
INFO - 2021-07-06 21:44:27,882 - settings - Setting database.pa

In [3]:
#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-07-06 21:44:29,526 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-07-06 21:44:29,527 - settings - Setting database.user to celiib
INFO - 2021-07-06 21:44:29,528 - settings - Setting database.password to newceliipass
INFO - 2021-07-06 21:44:29,547 - settings - Setting enable_python_native_blobs to True
INFO - 2021-07-06 21:44:30,541 - settings - Setting enable_python_native_blobs to True


In [4]:
import validation_utils as vu
import neuron_visualizations as nviz
import neuron_utils as nru
import proofreading_utils as pru
import time
import system_utils as su

In [5]:
minnie.AutoProofreadValidationSegmentMap4() & "segment_id=864691135593719339"

ver materialization version,nucleus_id nucleus id,segment_id the segment id in the proofread version,old_ver materialization version,old_segment_id the segment id in the proofread version--,finished_dendrite whether axon was finished cleaning,finished_axon whether axon was finished cleaning,extension_type,n_somas,n_soma_mergers,n_glia,large_splits,notes
88.00,554775,864691135593719339,0.08,864691136108720056,True,True,partial,1,0,0,0,


# Look at certain neuron after processing

In [6]:
segment_id = 864691135593719339
curr_table = (minnie.AutoProofreadValidationScore6() & dict(segment_id=segment_id))
decomp_path,validation_df = curr_table.fetch1("decomposition","validation_df")
neuron_obj = du.filepath_to_neuron_obj(decomp_path)

Decompressing Neuron in minimal output mode...please wait


In [7]:
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict="all")

Recieved another instance of Neuron class in init -- so just copying data

 Working on visualization type: mesh

 Working on visualization type: skeleton
No Synapses to plot


In [10]:
return_validation_df_revised

,synapse_id,synapse_type,auto_valid,synapse_x,synapse_y,synapse_z,manual_valid,category,old_segment_id,new_segment_id,euclidean_distance_to_nuclei
0,384279448,presyn,True,1252136,633752,838680,True,TP,864691136108720056,864691135593719339,127582
1,386239539,presyn,True,1252308,642028,838800,False,FP,864691136108720056,864691135593719339,135572
2,386239683,presyn,True,1253080,641440,835600,False,FP,864691136108720056,864691135593719339,134203
3,386239701,presyn,True,1253272,641136,835960,False,FP,864691136108720056,864691135593719339,133967
4,386239821,presyn,True,1254072,640432,831480,False,FP,864691136108720056,864691135593719339,132297
...,...,...,...,...,...,...,...,...,...,...,...
4556,456449855,postsyn,True,1382776,343736,771440,True,TP,864691136108720056,864691135593719339,206931
4557,513330591,postsyn,True,1292592,327208,923080,True,TP,864691136108720056,864691135593719339,219138
4558,513330627,postsyn,True,1292476,326844,923560,True,TP,864691136108720056,864691135593719339,219685
4559,396860706,postsyn,False,1280592,512320,906920,True,FN,864691136108720056,864691135593719339,101207


In [20]:
return_validation_df_revised = vu.load_df_val_from_numpy(validation_df)
filt_neuron = neuron_obj
vu.plot_validation_synapses(filt_neuron.segment_id,
                            validation_df = return_validation_df_revised,
                             synapse_dict = None,
                             synapses_type_to_plot = None,
                             synapses_type_to_not_plot = None,
                             verbose=False,
                             synapse_type = "presyn",
                             TP_color="yellow",
                            TN_color="aqua",
                            FP_color="black",
                            FN_color="orange",
                             synapse_scatter_size = 0.07,
                             
                             #for plotting the actual mesh parts to go along
                             
                             neuron_obj = filt_neuron,
                             plot_only_axon_skeleton = True,
                             error_mesh_color = "red",
                             valid_mesh_color = "green",
                             valid_skeleton_color = "black",
                             mesh_alpha = 0.3,)

Using the mesh from the neuron object


TP:yellow
TN:aqua
FP:black
FN:orange


In [ ]:
nviz.plot_

In [14]:
synapse_scatters = [np.array([[1252136, 633752, 838680],
       [1242728, 631752, 837680],
       [1287472, 635736, 722120],
       [1282904, 634112, 746760],
       [1300944, 641880, 705160],
       [1305008, 640912, 700720],
       [1307536, 645904, 693360],
       [1307432, 645072, 693320],
       [1318208, 651952, 675520],
       [1322616, 654208, 669800],
       [1314840, 650700, 679760],
       [1316632, 703852, 819240],
       [1328920, 657496, 659760],
       [1325448, 663304, 644800],
       [1326040, 662368, 647600],
       [1338344, 711664, 805000],
       [1325496, 706912, 812840],
       [1349344, 701032, 840760],
       [1350848, 703888, 829320],
       [1345632, 711864, 809840],
       [1347320, 708784, 819920],
       [1371776, 729792, 787440],
       [1358936, 721040, 796360],
       [1367768, 723472, 792320],
       [1395496, 738896, 775600],
       [1410876, 738596, 761840],
       [1425752, 738096, 758720],
       [1427960, 737584, 757080],
       [1447564, 737496, 750920]], dtype=object), np.array([[1207664, 512916, 816080],
       [1201136, 513808, 787160],
       [1193592, 565232, 762000],
       [1212360, 516656, 789120],
       [1225968, 518968, 789000],
       [1242080, 416440, 778240],
       [1240712, 524568, 757840],
       [1237116, 379108, 791200],
       [1252416, 426448, 780520],
       [1254016, 426184, 781160],
       [1275296, 494648, 789960],
       [1272352, 586600, 814840],
       [1271504, 595032, 823720],
       [1273160, 590368, 821880],
       [1274856, 581832, 817720],
       [1275296, 581312, 817600],
       [1265912, 603040, 826240],
       [1282088, 501664, 857400],
       [1280456, 503832, 848640],
       [1287856, 522336, 823120],
       [1291192, 567648, 817080],
       [1284908, 510312, 826880],
       [1287560, 518192, 826720],
       [1290536, 573576, 814280],
       [1285056, 578056, 822400],
       [1277096, 461024, 781400],
       [1285080, 632420, 792200],
       [1285328, 633248, 790960],
       [1285696, 624064, 792120],
       [1285800, 627424, 791800],
       [1285916, 624084, 792480],
       [1286136, 629164, 792680],
       [1278516, 611100, 793880],
       [1279448, 615592, 797000],
       [1288728, 582616, 884760],
       [1281032, 604168, 801960],
       [1281912, 605968, 800920],
       [1282864, 609688, 800320],
       [1291584, 579272, 874480],
       [1286628, 526460, 814320],
       [1286724, 526744, 815040],
       [1280704, 504384, 836760],
       [1291680, 586560, 808240],
       [1304944, 505136, 799440],
       [1292044, 572928, 816200],
       [1293040, 570184, 815880],
       [1306584, 582056, 812920],
       [1322288, 511312, 781600],
       [1315640, 621872, 793480],
       [1316280, 621644, 793240],
       [1316496, 628224, 792840],
       [1318240, 627600, 791920],
       [1318392, 628572, 790680],
       [1310392, 607368, 795200],
       [1314544, 617752, 793920],
       [1315328, 589024, 811720],
       [1319120, 589800, 810440],
       [1324120, 677644, 771840],
       [1327608, 595160, 807440],
       [1327844, 595100, 807440],
       [1328576, 595528, 806520],
       [1333656, 597848, 805720],
       [1337096, 598416, 804400],
       [1337304, 597392, 804840],
       [1344080, 532944, 743560],
       [1348852, 603396, 804240],
       [1368392, 365936, 787760],
       [1362048, 605880, 799560],
       [1363968, 605968, 799360],
       [1382704, 333392, 788560],
       [1374288, 611944, 797120],
       [1374448, 612000, 796440],
       [1384240, 622104, 796720]], dtype=object), np.array([[1252308, 642028, 838800],
       [1253080, 641440, 835600],
       [1253272, 641136, 835960],
       [1254072, 640432, 831480],
       [1255952, 639936, 829360],
       [1278048, 509816, 804920],
       [1288112, 530568, 821280],
       [1287000, 557752, 817480],
       [1287184, 517920, 824880],
       [1288780, 555040, 798600],
       [1280856, 572864, 808880],
       [1281336, 578224, 811920],
       [1281264, 575376, 812120],
       [1281360, 576144, 811960],
       [1281488, 581680, 823400],
       [1284472, 582072, 820560],
       [1287504, 573568, 820600],
       [1278992, 500832, 798120],
       [1282000, 521600, 813880],
       [1287880, 566152, 818720],
       [1411600, 737224, 750040]], dtype=object)]

In [16]:
nviz.plot_objects(scatters=synapse_scatters,
                  scatters_colors=["red","blue","yellow"])

# Getting all of the scores

In [ ]:
minnie.AutoProofreadValidationScore6()

In [ ]:
minnie,_ = du.configure_minnie_vm()
validation_dfs = minnie.AutoProofreadValidationScore6().fetch("validation_df")
validation_dfs = [vu.load_df_val_from_numpy(k) for k in validation_dfs]
validation_dfs

In [ ]:
import pandas_utils as pu
total_validation_df = pu.concat(validation_dfs)
total_validation_df
vu.synapse_validation_df_to_score_plot(total_validation_df)

# Looking at a certain neuron

In [ ]:
#segment_id = 864691135945413668
#segment_id = 864691136951664863
segment_id = 864691135740225387
segment_id = 864691136008425132

segment_id = 864691136311791677
neuron_objs,neuron_split_idxs = du.decomposition_with_spine_recalculation(segment_id)

In [ ]:
neuron_obj = neuron_objs[0]
print(neuron_obj.axon_limb_branch_dict)
nviz.plot_axon(neuron_obj)

In [ ]:
inh_exc_class = "excitatory"
plot_limb_branch_filter_with_disconnect_effect = True
perform_axon_classification = False
verbose = True
filtered_neuron,filtering_info = pru.proofread_neuron_class_predetermined(neuron_obj,
        inh_exc_class=inh_exc_class,
        perform_axon_classification=False,

        plot_limb_branch_filter_with_disconnect_effect = plot_limb_branch_filter_with_disconnect_effect,


        plot_final_filtered_neuron = False,

        #arguments for the axon high fidelity:
        plot_new_axon_limb_correspondence = False,
        plot_new_limb_object = False,
        plot_final_revised_axon_branch = False,

        verbose = False,
        verbose_outline = verbose,
        #filter_list = filter_list
        filter_list = pru.v6_exc_filters(),
        #filter_list = [pru.exc_high_degree_branching_filter_v6],
        high_fidelity_axon_on_excitatory = perform_axon_classification,
                                                                             )

In [ ]:
debug

In [ ]:
d_skeletons_resized = [sk.resize_skeleton_branch(k,segment_width=50)
                          for k in downstream_skeletons]
joining_endpoint_1 = np.array([822357. , 785765.2, 939390.7])
joining_endpoint_1
comparison_distance = 2000

In [ ]:
d_skeletons_restricted = [sk.restrict_skeleton_to_distance_from_coordinate(k,
                                             coordinate = joining_endpoint_1,
                                             distance_threshold = comparison_distance+1,)
                              for k in d_skeletons_resized]

In [ ]:
import system_utils as su
downstream_skeletons = su.decompress_pickle("downstream_skeletons")

In [ ]:
import skeleton_utils as sk
sk.shared_coordiantes(list(downstream_skeletons),
                             return_one=True)

In [ ]:
nviz.plot_objects(skeletons=downstream_skeletons,
                 scatters=[np.array([822357. , 785765.2, 939390.7])])

In [ ]:
sk.find_skeleton_endpoint_coordinates(downstream_skeletons[0]),sk.find_skeleton_endpoint_coordinates(downstream_skeletons[1])

In [ ]:
limb_obj = neuron_obj[neuron_obj.axon_limb_idx]
limb_obj

In [ ]:
nru.upstream_endpoint(limb_obj,402)

In [ ]:
nru.downstream_endpoint(limb_obj,402)

In [ ]:
nru.restrict_skeleton_from_start_plus_offset_upstream(limb_obj,
                                                      402,
                                                      verbose=True
                                                     )

In [ ]:
import system_utils as su

In [ ]:
restrict_skeleton_from_start_plus_offset_downstream

In [ ]:
test_sk = su.decompress_pickle("test_sk.pbz2")

In [ ]:
.endpoints

In [ ]:
branch_obj = neuron_obj[0][20]
nviz.plot_objects(skeletons=[branch_obj.skeleton],
                  meshes=[branch_obj.mesh],
                 scatters=[branch_obj.endpoints])

In [ ]:
import error_detection as ed
arg_dict = dict(child_width_maximum = 75,
                                        parent_width_maximum = 75,
                                         axon_only = True,
                                         error_if_web_is_none=True,
                                         web_size_threshold=120,
                                        web_size_type="ray_trace_median",
                                        web_above_threshold = True,
                                           ignore_if_child_mesh_not_touching = True )
ed.webbing_t_errors_limb_branch_dict(neuron_obj,verbose = True,plot_web_errors = True,**arg_dict)

In [ ]:
limb_obj = neuron_obj[0]

In [ ]:
import trimesh_utils as tu
import numpy_utils as nu

In [ ]:
downstream_nodes = [10,148]
upstream_node = 147

upstream_node = 13


downstream_nodes_mesh_connected(limb_obj,147,
                               verbose=True)

In [ ]:
intersect_array = nu.intersect2d([],np.array([[0,1],[0,2]]))
intersect_array

In [ ]:
downstream_nodes

In [ ]:
meshes=[limb_obj[k].mesh for k in downstream_nodes + [upstream_node]]
nviz.plot_objects(meshes=meshes)

In [ ]:
nviz.plot_branch_groupings(limb_obj,
                          [[upstream_node],downstream_nodes],)

In [ ]:
minnie.AutoProofreadValidationSegmentMap4() & dict(segment_id=864691136951664863)

# Runnig the validation

In [ ]:
"""
Psueodocode: 
1) Pull down the segment ids of the valid neurons

For each neuron
1) Dwonload the neuron object
2) Run the Proofreading
3) Print out the validation plot
4) Save the validation df

"""

In [ ]:
"""
Pseudocode: 
1) Check if there are multiple somas associated with that segment

"""

In [ ]:
axon_segs = dj.U("segment_id") & (minnie.DecompositionAxon() 
                      & minnie.AutoProofreadValidationSegment4() - minnie.AutoProofreadValidationSegmentExclude4() )
axon_segs

In [ ]:
"""
Pseudocode: 
1) Find the coordinates of the nucleus for that new segment
2) Make sure that same number of DecompositionAxon objects as in Decomposition
3) Pick the neuron object that is closest and within a certain range of the nucleus
4) Download the neuron object
5) Run the proofreading
6) Save off the validation df
"""

In [ ]:
minnie.AutoProofreadValidationScore5()

In [ ]:
bad_postsyn_table = (minnie.AutoProofreadValidationScore5() & "post_fn>200")
bad_postsyn_table_filt =  (bad_postsyn_table * minnie.AutoProofreadValidationSegmentMap4()) & axon_segs.proj(old_segment_id="segment_id")
#bad_postsyn_seg_ids = bad_postsyn_table.fetch("segment_id")
bad_postsyn_table_filt.proj("post_fn")

In [ ]:
minnie.AutoProofreadValidationScore6()

In [ ]:
import datajoint as dj
dj.config["display.limit"] = 30
bad_presyn_table = (minnie.AutoProofreadValidationScore5() & "pre_fp>10")
bad_presyn_seg_ids = bad_presyn_table.fetch("segment_id")
bad_presyn_table

In [ ]:
# multi_soma_keys = (minnie.AutoProofreadValidationSegmentMap4() & "n_somas>1"
#                       & minnie.DecompositionAxon.proj(old_segment_id="segment_id")
#                   & dict(segment_id = 864691135415710906)).proj().fetch(as_dict=True)
# key = multi_soma_keys[0]
# key

In [ ]:
segment_id = 864691135117980637
key = dict(segment_id=segment_id)

In [ ]:
verbose = True

segment_id=key["segment_id"]
print(f"\n-- Working on neuron {segment_id}---")

segment_map_dict = (minnie.AutoProofreadValidationSegmentMap4() & key).fetch1()

#1) Find the coordinates of the nucleus for that new segment
nucleus_id = segment_map_dict["nucleus_id"]
nuc_center_coords = du.nuclei_id_to_nucleus_centers(nucleus_id)
if verbose:
    print(f"nuc_center_coords = {nuc_center_coords}")

#2) Make sure that same number of DecompositionAxon objects as in Decomposition
old_segment_id = segment_map_dict["old_segment_id"]
if verbose:
    print(f"old_segment_id = {old_segment_id}")
    
search_key = dict(segment_id=old_segment_id)
n_somas = len(minnie.BaylorSegmentCentroid() & search_key)
n_decomp_axon = len(minnie.DecompositionAxon() & search_key)
if verbose:
    print(f"# of somas = {n_somas} and # of DecompositionAxon = {n_decomp_axon}")
    

if n_somas != n_decomp_axon:
    raise Exception(f"# of somas = {n_somas} NOT MATCH # of DecompositionAxon = {n_decomp_axon}")
    
#3) Pick the neuron object that is closest and within a certain range of the nucleus
neuron_objs,split_idxs = du.decomposition_with_spine_recalculation(old_segment_id)
if n_somas > 1:
    """
    Finding the closest soma:
    1) For each neuron object get the mesh center of the soma object
    2) Find the distance of each from the nucleus center
    3) Find the arg min distance and make sure within threshold
    4) Mark the current neuron and the current split index
    """
    nuclei_distance_threshold = 15000

    soma_center_coords = [k["S0"].mesh_center for k in neuron_objs]
    soma_distances = [np.linalg.norm(k-nuc_center_coords) for k in soma_center_coords]
    min_dist_arg = np.argmin(soma_distances)
    min_dist = soma_distances[min_dist_arg]

    if verbose:
        print(f"soma_distances = {soma_distances}")
        print(f"min_dist_arg = {min_dist_arg}, with min distance = {min_dist}")

    if min_dist > nuclei_distance_threshold:
        raise Exception(f"min_dist ({min_dist}) larger than nuclei_distance_threshold ({nuclei_distance_threshold})")

    neuron_obj = neuron_objs[min_dist_arg]
    split_index = split_idxs[min_dist_arg]

    if verbose:
        print(f"Winning split_index = {split_index}")
else:
    split_index = split_idxs[0]
    neuron_obj = neuron_objs[0]
    
    
import proofreading_utils as pru
import validation_utils as vu
(filt_neuron,
     return_synapse_df_revised,
     return_synapse_df_errors,
    return_validation_df_revised,
    return_validation_df_extension) =  vu.filtered_neuron_score(neuron_obj = neuron_obj,   
                        filter_list = pru.v5_exc_filters(),
                        plot_limb_branch_filter_with_disconnect_effect = True,
                        verbose = True,
                        plot_score=False,
                        nucleus_id = nucleus_id,
                        return_synapse_df_errors=True,
                        return_validation_df_extension = True,                                        
                        split_index=split_index)
    

In [ ]:
vu.synapse_validation_df_to_score_plot(return_validation_df_revised)

In [ ]:
nviz.plot_axon(filt_neuron)

In [ ]:
# import neuron_visualizations as nviz
# nviz.visualize_neuron(filt_neuron,
#                       limb_branch_dict="all")

In [ ]:
minnie.AutoProofreadValidationSegmentMap4() & key

In [ ]:
filt_neuron.segment_id

In [ ]:
vu.plot_validation_synapses(filt_neuron.segment_id,
                            validation_df = return_validation_df_revised,
                             synapse_dict = None,
                             synapses_type_to_plot = None,
                             synapses_type_to_not_plot = None,
                             verbose=False,
                             synapse_type = "presyn",
                             TP_color="yellow",
                            TN_color="aqua",
                            FP_color="black",
                            FN_color="orange",
                             synapse_scatter_size = 0.15,
                             
                             #for plotting the actual mesh parts to go along
                             
                             neuron_obj = filt_neuron,
                             plot_only_axon_skeleton = True,
                             error_mesh_color = "red",
                             valid_mesh_color = "green",
                             valid_skeleton_color = "black",
                             mesh_alpha = 0.3,)

# How to attach the synapses

In [ ]:
beginning_direct_connections = du.segment_id_to_synapse_table(filt_neuron.segment_id,
                                                                         validation=True)
beginning_direct_connections

In [ ]:
864691135584074360

# Debugging the Stitch Distance

In [ ]:
incorp_floating_pieces = nru.non_soma_touching_meshes_stitched(filt_neuron)
incorp_floating_pieces

In [ ]:
nviz.plot_objects(meshes = incorp_floating_pieces)

In [ ]:
nru.filter_branches_by_restriction_mesh(filt_neuron[2],
                                       restriction_mesh=incorp_floating_pieces[0])

In [ ]:
[k for k in dir(nru) if "mesh" in k]

In [ ]:
nru.find_parent_child_skeleton_angle(filt_neuron[2],68,
                                     comparison_distance=5000,
                                     offset=0,
                                    plot_extracted_skeletons=True)

In [ ]:
nru.upstream_node(filt_neuron[2],68)

In [ ]:
nviz.plot_limb_path(filt_neuron[2],[68])

In [ ]:
nviz.visualize_neuron_specific_limb(filt_neuron,
                                   filt_neuron.axon_limb_idx)

In [ ]:
minnie.AutoProofreadStats5()

In [ ]:
du.plot_proofread_neuron(864691135517422218,
                         plot_proofreading_synapses=True,
                        plot_errored_faces=True)

# Checking the long Axons

In [ ]:
du.proofreading_neurons_table() & "axon_length>2000" & "spine_category='densely_spined'"

In [ ]:
du.plot_proofread_neuron(864691134917429770,
                        plot_proofreading_synapses=True)

In [ ]:
import ipyvolume as ipv
ipv.gcf()

In [ ]:
ipv.pylab.save("864691134917429770_proof_v5.html")

In [ ]:
du.plot_proofread_mesh_skeleton(864691134917429770)

In [ ]:
minnie,_ = du.configure_minnie_vm()
from importlib import reload
du = reload(du)